In [12]:
import math
import random
from functools import partial
from typing import Callable, TypeAlias

In [13]:
Individual: TypeAlias = list[int]
Population: TypeAlias = list[Individual]

def generate_individual(length: int) -> Individual:
    random.shuffle(individual := list(range(1, length + 1)))
    return individual


def generate_population(pop_size: int, individual_length: int) -> Population:
    return [generate_individual(individual_length) for _ in range(pop_size)]

In [14]:
FitnessFunc: TypeAlias = Callable[[Individual], int]

def fitness(individual: Individual) -> int:
    clashes = 0
    for i in range(len(individual) - 1):
        for j in range(i + 1, len(individual)):
            if abs(individual[j] - individual[i]) == j - i:
                clashes += 1
    return clashes

In [15]:
IndividualPair: TypeAlias = tuple[Individual, Individual]
SelectionFunc: TypeAlias = Callable[[Population, FitnessFunc], IndividualPair]

def roulette_selection(population: Population, fitness: FitnessFunc) -> IndividualPair:
    parents = random.choices(
        population=population,
        weights=[fitness(individual) for individual in population],
        k=2,
    )
    return parents[0], parents[1]

In [16]:
CrossoverFunc: TypeAlias = Callable[[IndividualPair], IndividualPair]

def ordered_crossover(parents: IndividualPair) -> IndividualPair:
    parent_a, parent_b = parents
    split_idx = random.randint(1, len(parent_a) - 1)
    offspring_x = parent_a[:split_idx] + list(
        filter(lambda pos: pos not in parent_a[:split_idx], parent_b)
    )
    offspring_y = parent_b[:split_idx] + list(
        filter(lambda pos: pos not in parent_b[:split_idx], parent_a)
    )
    return offspring_x, offspring_y

In [17]:
MutationFunc: TypeAlias = Callable[[Individual, float], Individual]

def swap_mutation(individual: Individual, probability: float) -> Individual:
    if random.random() <= probability:
        pos1 = random.randint(0, len(individual) - 1)
        pos2 = random.randint(0, len(individual) - 1)
        individual[pos1], individual[pos2] = individual[pos2], individual[pos1]
    return individual

In [18]:
def compute_next_generation(population: list[Individual], mutation_prob: float = 0.3, n_elites: int = 10):
    next_generation = population[:n_elites]
    for _ in range(int((len(population) - n_elites) / 2)):
        parents = roulette_selection(population, fitness)
        offspring = ordered_crossover(parents)
        next_generation += map(
            partial(swap_mutation, probability=mutation_prob), offspring
        )
    return next_generation

In [19]:
def run_evolution(pop_size: int, individual_length: int, fitness_limit: int = 0,
    mutation_prob: float = 0.3, n_iter: int = 1000, n_elites: int = 10) -> Individual:

    population = generate_population(pop_size, individual_length)
    for i in range(n_iter):
        population = sorted(population, key=fitness)
        print(f"Generation {i} - Best fitness {fitness(population[0])}")
        if fitness(population[0]) <= fitness_limit:
            break
        population = compute_next_generation(population, mutation_prob, n_elites)
    return sorted(population, key=fitness)[0]

In [20]:
random.seed(123)  # set the seed for reproducibility

In [ ]:
best_solution = run_evolution(pop_size=100, individual_length=15)

In [ ]:
def view_chessboard(individual: Individual) -> None:
    print(f"\nNumber of possible solution {math.factorial(len(individual)):,}")
    print(f"Solution: {individual}\n")

    for row, _ in enumerate(individual):
        for col, _ in enumerate(individual):
            cell = "[Q]" if individual[col] - 1 == row else "[ ]"
            print(cell, end="")
        print()


view_chessboard(best_solution)